    from IPython import get_ipython; ip = get_ipython()
    %reload_ext pidgin.tangle

`import pidgin.tangle` modifies the `get_ipython().input_transformer_manager` to accept __Markdown__ source.  The `pidgin.tangle` module
exports:
    
* `pidgin.tangle.markdown_to_python` - is a semi-lossless __Markdown__ to __Python__ converter.
* `pidgin.tangle.markdown_to_statements` - extracts inline code from __Markdown__ text.

In [1]:
    import ast, mistune, textwrap, functools, itertools, IPython
    __all__ = 'markdown_to_python', 'markdown_to_statements'

`quote` wrotes non code objects in triple ticks.

In [2]:
    def quote(str, punc=''):
        str, leading_ws = ''.join(str), []
        lines = str.splitlines(True)
        _ = '"""'
        if _ in str: _ = "'''"
        if not str.strip(): _ = punc = ''
        while lines and (not lines[0].strip()): leading_ws.append(lines.pop(0))    
        str = ''.join(lines)
        end = len(str.rstrip())
        str, ending_ws = str[:end], str[end:]
        if str and str.endswith(_[0]): str += ' '                    
        return F"{''.join(leading_ws)}{_}{str}{_}{punc}{ending_ws}"

`get_first_line` get the first non-`iter`able strings in `lines`

In [3]:
    def get_first_line(lines, line=''):
        for line in lines or ['']: 
            if line.strip(): break
        return line

`get_line_indent` computes the indent of a string.

In [4]:
    def get_line_indent(line):  return len(line) - len(line.lstrip())

The `Markdown` transformer must combine code and non-code.  It must handle logic to properly indent tangled code and non-code.

In [5]:
    class Markdown(mistune.Markdown):
        def render(self, text):
            """Initialize the """
            self.renderer.original = ''.join(text).splitlines()
            self.renderer.final, self.renderer.buffer, self.renderer.min_indent = [], [], 4
            self.parse(text)
            self.renderer.block_code('', None, punc=';')
            return '\n'.join(self.renderer.final)

The __Lexer__ s only consider coarse features of the markdown spec.  

In [6]:
    class BlockLexer(mistune.BlockLexer): 
        list_rules = default_rules = [
            'newline', 'fences', 'block_code', 'block_quote', 'list_block', 'block_html', 'paragraph', 'text']
    class InlineLexer(mistune.InlineLexer):  
        inline_html_rules = default_rules = ['text']

In [7]:
    class Renderer(mistune.Renderer):
        def text(self, text): return text
        def newline(self):
            self.buffer += ['']
            return ''
        
        def paragraph(self, text):
            line, lines = '', text.splitlines()
            while lines and not lines[-1].strip():  lines.pop()
            while lines:
                line = lines.pop(0)
                while self.original and line not in self.original[0]:
                    self.buffer += [self.original.pop(0)]
            if self.original and line in self.original[0]:  self.buffer += [self.original.pop(0)]
            return ''

        def block_code(self, text, lang, punc=''): 
            first_line = get_first_line(text.splitlines())
            
            while self.original and (lang or first_line) not in self.original[0]:   
                self.buffer += [self.original.pop(0)]
                
            while self.original and not self.original[0].strip(): self.buffer += [self.original.pop(0)]
                
            code, lines = [], text.splitlines()
            
            if lang:
                lines = [lang] + lines            
                if not lang.startswith('%%'): return self.paragraph('\n'.join(lines))

            while lines and not lines[0].strip(): lines.pop(0)
            
            if self.original:
                while lines:
                    line = lines.pop(0)
                    while line not in self.original[0]: code += [self.original.pop(0)]
                    code += [self.original.pop(0)]
                    
            code = '\n'.join(code + ['']).lstrip('```')
            
            if code.lstrip().startswith('%%'):
                code = IPython.core.inputtransformer2.TransformerManager().transform_cell(code)
                
            body = '\n'.join(self.buffer + [''])
            
            last_line = get_first_line(reversed(self.final))
            prior_indent = get_line_indent(last_line)

            definition, returns = last_line.rstrip().endswith(':'), last_line.lstrip().startswith('return')

            this_indent = get_line_indent(get_first_line(code.splitlines()))
            
            if this_indent < self.min_indent:
                code = textwrap.indent(code, ' '*(self.min_indent-this_indent))
                this_indent = get_line_indent(get_first_line(code.splitlines()))

            if body.strip() and not self.min_indent: self.min_indent = this_indent
            indent = max(self.min_indent, (returns and min or max)(prior_indent, this_indent))
            
            
            
            if not (last_line.endswith('"""') or last_line.endswith("'''")): 
                body = quote(body, punc)
            
            body = textwrap.indent(body, ' '*indent)
            
            if definition and prior_indent == indent: 
                body = ' '*4+body
                
            self.buffer = self.final.extend(
                body.splitlines() + code.splitlines()) or []
            return ''            

## Exports

`markdown_to_python` converts __Markdown__ to __Python__ in a semi lossless way.

In [8]:
    def markdown_to_python(str)->"Valid Python Source": 
        renderer = Renderer()
        if str.lstrip().startswith('%%'): return str
        return Markdown(renderer=renderer, 
            inline=InlineLexer(renderer, mistune.InlineGrammar()), 
            block=BlockLexer(mistune.BlockGrammar())).render(str)

`InlineExpressionsMarkdown` is a special `mistune.Markdown` transformer that records information in __code span__ statatements.

    print(markdown_to_python('''
    ## Special `pidgin.inspector` indentifiers.

    - __object?__ or __object??__ - one question mark returns basic info, two question marks returns the source with the `object` info.  Question marks supercede all other 
    delimiters.

            question_marks = "([\S]+)\?{1}|\?{1}([\S]+)"
    - __{{brace[0]}}"expression or statement"{{brace[1]}}__ - author's recieve a preview of the `jinja2` expression.

            braced = "(\{){2}([^(\?}{2})]+)\}{2}"
    - __{{tick}}"expression or statement"{{tick}}__ - executes the inline code and displays a preview.  _Statements and expressions in inline code
    are not rendered by `pidgin`, but intermediate representations assist in authoring `pidgin` documents._

            ticked = "`([^\?]+)`"

    The conditions for a regular expression used to discover substrings.

            inline_code_objects = re.compile(F"{'|'.join((braced, ticked, question_marks))}")

    When no `inline_code_objects` are found the `pidgin.inspector` defaults to a __Markdown__ preview.'''))

In [9]:
    class InlineExpressionsMarkdown(mistune.Markdown):
        def render(self, text):
            self.renderer.expressions = []
            super().render(text)
            return self.renderer.expressions

`InlineExpressionsRenderer` is a special `mistune.Renderer` transformer that records expressions in `InlineExpressionsRenderer.codespan`s.

In [10]:
    class InlineExpressionsRenderer(mistune.Renderer):
        def codespan(self, text): 
            self.expressions.append(text)
            return super().text(text)

In [11]:
    def markdown_to_statements(str):
        return InlineExpressionsMarkdown(renderer=InlineExpressionsRenderer()).render(str)

In [12]:
    def transformer(lines: "that end with a newline."): 
        # Always add a new line
        return markdown_to_python(''.join(lines + ['\n'])).splitlines(1)

In [13]:
    def load_ipython_extension(ip=None):
        (ip or IPython.get_ipython()).input_transformer_manager.cleanup_transforms.insert(0, transformer)

    def unload_ipython_extension(ip=None):
        ip = ip or IPython.get_ipython()
        ip.input_transformer_manager.cleanup_transforms = [
            object for object in ip.input_transformer_manager.cleanup_transforms
            if object is not transformer]
    if __name__ == '__main__': load_ipython_extension()